In [2]:
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from datetime import datetime, timedelta

In [10]:
# 1. Simulate CA root key (self-signed)
ca_key = ec.generate_private_key(ec.SECP256R1())
ca_name = x509.Name([
    x509.NameAttribute(NameOID.COMMON_NAME, u"My CA"),
    x509.NameAttribute(NameOID.COUNTRY_NAME, u"IN"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"Tamil Nadu"),
    x509.NameAttribute(NameOID.LOCALITY_NAME, u"Chennai"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"IIT Madras"),
    x509.NameAttribute(NameOID.COMMON_NAME, u"ee21b128@smail.iitm.ac.in"),
])

ca_cert = (
    x509.CertificateBuilder()
    .subject_name(ca_name)
    .issuer_name(ca_name)
    .public_key(ca_key.public_key())
    .serial_number(x509.random_serial_number())
    .not_valid_before(datetime.utcnow())
    .not_valid_after(datetime.utcnow() + timedelta(days=365))
    .add_extension(x509.BasicConstraints(ca=True, path_length=None), critical=True)
    .sign(ca_key, hashes.SHA256())
)


In [4]:
# Function to create a user certificate signed by CA
def generate_user_cert(common_name: str):
    user_key = ec.generate_private_key(ec.SECP256R1())
    user_name = x509.Name([
        x509.NameAttribute(NameOID.COMMON_NAME, common_name),
    ])

    user_csr = (
        x509.CertificateSigningRequestBuilder()
        .subject_name(user_name)
        .sign(user_key, hashes.SHA256())
    )

    user_cert = (
        x509.CertificateBuilder()
        .subject_name(user_csr.subject)
        .issuer_name(ca_cert.subject)
        .public_key(user_key.public_key())
        .serial_number(x509.random_serial_number())
        .not_valid_before(datetime.utcnow())
        .not_valid_after(datetime.utcnow() + timedelta(days=90))
        .sign(ca_key, hashes.SHA256())
    )

    return user_key, user_cert

In [5]:
# Generate certificates for Alice and Bob
alice_key, alice_cert = generate_user_cert("Alice")
bob_key,   bob_cert   = generate_user_cert("Bob")


In [6]:
# 2. Each party extracts peer's public key from certificate
alice_pub = alice_cert.public_key()
bob_pub   = bob_cert.public_key()

In [7]:
# 3. Derive ECDH shared secret
alice_shared = alice_key.exchange(ec.ECDH(), bob_pub)
bob_shared   = bob_key.exchange(ec.ECDH(), alice_pub)
assert alice_shared == bob_shared

In [8]:
# 4. Derive a symmetric key via HKDF
hkdf = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b"user handshake",
)
symmetric_key = hkdf.derive(alice_shared)

In [9]:
# Serialize certificate to PEM for distribution
alice_cert_pem = alice_cert.public_bytes(serialization.Encoding.PEM)
bob_cert_pem   = bob_cert.public_bytes(serialization.Encoding.PEM)

print("Alice Certificate:\n", alice_cert_pem.decode())
print("Bob Certificate:\n", bob_cert_pem.decode())
print("Derived symmetric key (hex):", symmetric_key.hex())

Alice Certificate:
 -----BEGIN CERTIFICATE-----
MIIBHzCBxqADAgECAhR0GtlY3hu+9yniNeiBoay5OOZ77zAKBggqhkjOPQQDAjAQ
MQ4wDAYDVQQDDAVNeSBDQTAeFw0yNTA1MDIwOTA2MzVaFw0yNTA3MzEwOTA2MzVa
MBAxDjAMBgNVBAMMBUFsaWNlMFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEGkNe
1IZN3x5kW3ygJe53ksZCg8mJbMWDh86N1ODOCmE1YbTTMI9X/peWMg8S8TQffGHl
BlWUux2P+80iHiuySjAKBggqhkjOPQQDAgNIADBFAiBqztj8sNDbUNTbrlsXb7/P
+S45gq53XLoOn2q9+xh7UwIhAOiWlRV7eQvt3+QQfmty2oYMgalb5sPansdeS4uN
uykV
-----END CERTIFICATE-----

Bob Certificate:
 -----BEGIN CERTIFICATE-----
MIIBHTCBxKADAgECAhRP0qUN/ILg3rjq3oGIMTp5a9KXuTAKBggqhkjOPQQDAjAQ
MQ4wDAYDVQQDDAVNeSBDQTAeFw0yNTA1MDIwOTA2MzVaFw0yNTA3MzEwOTA2MzVa
MA4xDDAKBgNVBAMMA0JvYjBZMBMGByqGSM49AgEGCCqGSM49AwEHA0IABNOKzyM4
70jHAwMvPOiJvsIdGY8THVdE7r2hsem2L2x1ulNUocduXugx/3TJ66NrMoHXMWC0
lrxiKroiIEBckx4wCgYIKoZIzj0EAwIDSAAwRQIgD98d4aP4euyGS6lqq/LdqGBC
l6TVxhQI9Bn74XGWh28CIQCVq5LCFzQZW0UlhtShkyZ+Gd/ZcTP7z/fgqmeyUyRm
yw==
-----END CERTIFICATE-----

Derived symmetric key (hex): 5286e99b3942b510fdbe53f2f0b71df30